In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
path = os.path.join(f"/home/stano/Project/Sensor", "Gas_Sensor/")
os.chdir(path)
files = sorted(os.listdir())

In [30]:
for i in range(len(files)):
    df = pd.read_csv(f'/home/stano/Project/Sensor/Gas_Sensor/{files[i]}')
    col = {i : i.split()[0] for i in df.columns}
    df.rename(columns=col, inplace=True)
    
    
    df_actual = df[(df["Flow"] >=235) & (df["Flow"] <=246)]
    df_actual = df_actual[df_actual["Temperature"] >= 21]
    
    df_actual["Heater"] = df_actual["Heater"].apply(lambda x: round(x,1))
    df_actual["Flow"] = df_actual["Flow"].apply(lambda x: int(x))
    df_actual["Humidity"] = df_actual["Humidity"].apply(lambda x: int(x))
    df_actual.drop("Time", axis =1, inplace =True)
    
    df_actual["CO"].where(~((df["CO"]>=0) & (df["CO"]<10)),"Low", inplace=True)
    #df_actual["CO"].where(~((df["CO"]>=6) & (df["CO"]<13)),"Moderate", inplace=True)
    df_actual["CO"].where(~(df["CO"]>=10),"High", inplace=True)
    
    df_actual["Sensor_1"] = (df_actual["R1"] + df_actual["R2"] + df_actual["R3"] + df_actual["R4"] + df_actual["R5"] + df_actual["R6"] + df_actual["R7"])/7
    df_actual["Sensor_2"] = (df_actual["R8"] + df_actual["R9"] + df_actual["R10"] + df_actual["R11"] + df_actual["R12"] + df_actual["R13"] + df_actual["R14"])/7
    
    df_ind = df_actual[["Humidity","Temperature", "Flow", "Heater", "Sensor_1", "Sensor_2"]]
    df_dep = df_actual["CO"]
    
    exec(f"actual_{i+1} = df_actual")
    exec(f"df_ind{i+1} = df_ind")
    exec(f"df_dep{i+1} = df_dep")

In [5]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
def optimization(X_train,y_train):
    
    l1 = list()
    for i in range(5,110,10):
        for j in range(30,130,10):
            d1 = dict()
            boost = GradientBoostingClassifier(learning_rate=i/1000,n_estimators=j)
            boost.fit(X_train,y_train)
            d1["learning_rate"] = i/1000
            d1["n_estimators"] = j
            d1["train_score"] = boost.score(X_train,y_train)
            
            for k in range(1,14):
                exec(f"d1['testscore_day{k}'] = boost.score(df_ind{k},df_dep{k})")

            l1.append(d1)

    return l1

In [10]:
from cassandra.cluster import Cluster
clustering = Cluster()
session = clustering.connect()

In [11]:
session.execute("create keyspace if not exists model with replication = {'class': 'SimpleStrategy', 'replication_factor': '1'} and durable_writes = true")

In [18]:
def load_optimization():
    
    optim = pd.DataFrame(optimization(df_ind2,df_dep2))
    
    if "index" in list(optim.columns):
        optim.drop("index", axis= 1, inplace = True)
    optim = optim.reset_index()
    
    
    columns_schema = """id float, learning_rate float,n_estimator float, train_score float,
       testscore_day1 float, testscore_day2 float,testscore_day3 float,testscore_day4 float,
       testscore_day5 float, testscore_day6 float, testscore_day7 float, testscore_day8 float,
       testscore_day9 float, testscore_day10 float, testscore_day11 float,
       testscore_day12 float,testscore_day13 float, primary key (id)"""
    
    session.execute(f"create table if not exists model.optim ({columns_schema})")
    
    
    columns = """id, learning_rate, n_estimator, train_score,
       testscore_day1, testscore_day2, testscore_day3, testscore_day4,
       testscore_day5, testscore_day6, testscore_day7, testscore_day8,
       testscore_day9, testscore_day10, testscore_day11,
       testscore_day12, testscore_day13"""
    
    for i in range(len(optim)):
        row = tuple(optim.iloc[i].values)
        session.execute(f"""insert into model.optim ({columns}) values {row}""")
        

In [19]:
load_optimization()

In [43]:
df = pd.DataFrame(list(session.execute("select * from model.optim")))

In [46]:
df["id"] = df["id"].astype("int")

In [47]:
df.sort_values(by="id", inplace=True)
df.set_index("id", inplace=True)

In [48]:
df.insert(2,"test_mean",np.mean(df.iloc[:,2:15], axis=1))

In [49]:
df.insert(3,"Deviation",np.std(df.iloc[:,3:16], axis=1))

In [50]:
df.head()

,learning_rate,n_estimator,test_mean,Deviation,testscore_day1,testscore_day10,testscore_day11,testscore_day12,testscore_day13,testscore_day2,testscore_day3,testscore_day4,testscore_day5,testscore_day6,testscore_day7,testscore_day8,testscore_day9,train_score
id,,,,,,,,,,,,,,,,,,
0,0.005,30.0,0.657626,0.028930,0.648269,0.667352,0.564298,0.654727,0.660468,0.676410,0.662750,0.643783,0.670594,0.674609,0.676963,0.675883,0.673025,0.676410
1,0.005,40.0,0.731177,0.032070,0.726891,0.742287,0.630026,0.728944,0.734350,0.754307,0.740725,0.703819,0.743534,0.750588,0.752255,0.751356,0.746224,0.754307
2,0.005,50.0,0.760626,0.033819,0.751193,0.773577,0.651751,0.764052,0.762055,0.782179,0.769075,0.738752,0.776067,0.777164,0.781696,0.780602,0.779977,0.782179
3,0.005,60.0,0.761523,0.033721,0.753126,0.774041,0.653226,0.764491,0.762204,0.783602,0.770751,0.738637,0.776551,0.778424,0.782847,0.781161,0.780742,0.783602
4,0.005,70.0,0.761642,0.033738,0.753126,0.774373,0.653257,0.764728,0.762339,0.783622,0.770849,0.738830,0.776764,0.778441,0.782972,0.781249,0.780790,0.783622


In [51]:
df.to_csv("/home/stano/Project/Sensor_Gas/optim.csv")

In [42]:
os.getcwd()

'/home/stano/Project/Sensor/Gas_Sensor'

In [63]:
df[df["test_mean"] > 0.798]

,learning_rate,n_estimator,test_mean,Deviation,testscore_day1,testscore_day10,testscore_day11,testscore_day12,testscore_day13,testscore_day2,testscore_day3,testscore_day4,testscore_day5,testscore_day6,testscore_day7,testscore_day8,testscore_day9,train_score
id,,,,,,,,,,,,,,,,,,
48,0.045,110.0,0.798161,0.040763,0.813573,0.788811,0.705472,0.774001,0.769818,0.855026,0.818662,0.736491,0.808986,0.827242,0.837891,0.833109,0.807017,0.855026
56,0.055,90.0,0.798002,0.041174,0.813640,0.788126,0.705872,0.772969,0.767878,0.855419,0.819627,0.734988,0.809565,0.826920,0.838074,0.833779,0.807172,0.855419
74,0.075,70.0,0.798505,0.042032,0.816087,0.787523,0.705263,0.772390,0.767445,0.857953,0.823094,0.734517,0.808637,0.827685,0.838923,0.834765,0.806279,0.857953


In [90]:
def optimization_leaf(X_train,y_train):
    
    l2 = list()
    for i in range(5,14,2):
        for j in range(50,550,100):
            d2 = dict()
            boost = GradientBoostingClassifier(learning_rate=0.075,n_estimators=70, max_depth=i, min_samples_split=j)
            boost.fit(X_train,y_train)
            d2["max_depth"] = i
            d2["min_sample_split"] = j
            d2["train_score"] = boost.score(X_train,y_train)
            
            for k in range(1,14):
                exec(f"d2['testscore_day{k}'] = boost.score(df_ind{k},df_dep{k})")

            l2.append(d2)

    return l2

In [91]:
def load_optimization_leaf():
    
    optim = pd.DataFrame(optimization_leaf(df_ind2,df_dep2))
    
    if "index" in list(optim.columns):
        optim.drop("index", axis= 1, inplace = True)
    optim = optim.reset_index()
    
    
    columns_schema = """id float, max_depth float,min_sample_split float, train_score float,
       testscore_day1 float, testscore_day2 float,testscore_day3 float,testscore_day4 float,
       testscore_day5 float, testscore_day6 float, testscore_day7 float, testscore_day8 float,
       testscore_day9 float, testscore_day10 float, testscore_day11 float,
       testscore_day12 float,testscore_day13 float, primary key (id)"""
    
    session.execute(f"create table if not exists model.optim_tree ({columns_schema})")
    
    
    columns = """id, max_depth, min_sample_split, train_score,
       testscore_day1, testscore_day2, testscore_day3, testscore_day4,
       testscore_day5, testscore_day6, testscore_day7, testscore_day8,
       testscore_day9, testscore_day10, testscore_day11,
       testscore_day12, testscore_day13"""
    
    for i in range(len(optim)):
        row = tuple(optim.iloc[i].values)
        session.execute(f"""insert into model.optim_tree ({columns}) values {row}""")
        

In [ ]:
load_optimization_leaf()

In [93]:
df_optim_tree = pd.DataFrame(list(session.execute("select * from model.optim_tree")))

In [96]:
df_optim_tree["id"] = df_optim_tree["id"].astype("int")
df_optim_tree.sort_values(by="id", inplace=True)
df_optim_tree.set_index("id", inplace=True)

In [99]:
df_optim_tree.insert(2,"test_mean",np.mean(df_optim_tree.iloc[:,2:15], axis=1))

In [103]:
df_optim_tree.insert(3,"Deviation",np.std(df_optim_tree.iloc[:,3:16], axis=1))

In [105]:
df_optim_tree.to_csv("/home/stano/Project/Sensor_Gas/optim_tree.csv")

In [106]:
df_optim_tree[df_optim_tree["test_mean"] > 0.80]

,max_depth,min_sample_split,test_mean,Deviation,testscore_day1,testscore_day10,testscore_day11,testscore_day12,testscore_day13,testscore_day2,testscore_day3,testscore_day4,testscore_day5,testscore_day6,testscore_day7,testscore_day8,testscore_day9,train_score
id,,,,,,,,,,,,,,,,,,


In [109]:
df_optim_tree["test_mean"]

id
0     0.763540
1     0.762179
2     0.762022
3     0.763125
4     0.766928
5     0.743566
6     0.744648
7     0.741166
8     0.746479
9     0.742484
10    0.737739
11    0.733152
12    0.733863
13    0.738121
14    0.729674
15    0.721938
16    0.721033
17    0.722028
18    0.722280
19    0.727975
20    0.710551
21    0.712524
22    0.713506
23    0.709111
24    0.714489
Name: test_mean, dtype: float64

In [122]:
df_dep13.value_counts()

High        116890
Low          90692
Moderate     87780
Name: CO, dtype: int64

In [ ]:
df = Grade  

In [17]:
from sklearn import ensemble

In [31]:
model = ensemble.GradientBoostingClassifier(n_estimators=70, learning_rate=0.075)

In [32]:
model.fit(df_ind2,df_dep2)

GradientBoostingClassifier(learning_rate=0.075, n_estimators=70)

In [33]:
df_dep2.unique()

array(['Low', 'High'], dtype=object)

In [34]:
l = list()
for i in range(1,14):
    d = dict()
    d["Day"] = "day" + str(i)
    exec(f"d['Score'] = model.score(df_ind{i},df_dep{i})")
    l.append(d)

In [36]:
pd.DataFrame(l)

,Day,Score
0,day1,0.893265
1,day2,0.909354
2,day3,0.879635
3,day4,0.868476
4,day5,0.890585
5,day6,0.892555
6,day7,0.894180
7,day8,0.895292
8,day9,0.895106
9,day10,0.891644
